In [3]:
from pmdarima.model_selection import RollingForecastCV
import numpy as np

# Customized cross validation with rolling window and XGboost
def custom_cross_val_predict(estimator, y, X=None, cv=None, verbose=0, averaging="mean", return_raw_predictions=False, initial=2555):
    """Generate cross-validated estimates for each input data point
    
    Parameters
    ----------
    estimator : tuple
        A tuple containing two estimators. The first estimator should be the ARIMA model
        and the second one should be the Random Forest model.

    y : array-like or iterable, shape=(n_samples,)
        The time-series array.

    X : array-like, shape=[n_obs, n_vars], optional (default=None)
        An optional 2-d array of exogenous variables.

    cv : BaseTSCrossValidator or None, optional (default=None)
        An instance of cross-validation. If None, will use a RollingForecastCV.
        Note that for cross-validation predictions, the CV step cannot exceed
        the CV horizon, or there will be a gap between fold predictions.

    verbose : integer, optional
        The verbosity level.

    averaging : str or callable, one of ["median", "mean"] (default="mean")
        Unlike normal CV, time series CV might have different folds (windows)
        forecasting the same time step. After all forecast windows are made,
        we build a matrix of y x n_folds, populating each fold's forecasts like
        so::

            nan nan nan  # training samples
            nan nan nan
            nan nan nan
            nan nan nan
              1 nan nan  # test samples
              4   3 nan
              3 2.5 3.5
            nan   6   5
            nan nan   4

        We then average each time step's forecasts to end up with our final
        prediction results.

    return_raw_predictions : bool (default=False)
        If True, raw predictions are returned instead of averaged ones.
        This results in a y x h matrix. For example, if h=3, and step=1 then:

            nan nan nan # training samples
            nan nan nan
            nan nan nan
            nan nan nan
            1   4   2   # test samples
            2   5   7
            8   9   1
            nan nan nan
            nan nan nan

        First column contains all one-step-ahead-predictions, second column all
        two-step-ahead-predictions etc. Further metrics can then be calculated
        as desired.

    Returns
    -------
    predictions : array-like, shape=(n_samples,)
        The predicted values.

    """

    def indexable(*iterables):
        """Internal utility to handle input types"""
        results = []
        for iterable in iterables:
            if not hasattr(iterable, "__iter__"):
                raise ValueError("Input {!r} is not indexable".format(iterable))
            results.append(iterable)
        return results

    def check_cv(cv, initial = 2555):
        """Internal utility to check cv"""
        if cv is None:
            cv = RollingForecastCV(initial=initial, step=1, h=1)
        return cv

    def check_endog(y, copy=True, preserve_series=False):
        """Internal utility to check endogenous variable"""
        from pmdarima.utils import check_endog
        return check_endog(y, copy=copy, preserve_series=preserve_series)

    def _check_averaging(averaging):
        """Internal utility to check averaging"""
        if averaging == "mean":
            return np.nanmean
        elif averaging == "median":
            return np.nanmedian
        elif callable(averaging):
            return averaging
        else:
            raise ValueError("Unknown averaging method: {}".format(averaging))

    def _fit_and_predict(fold, estimator_tuple, y, X, train, test, verbose=1):
        """Internal utility to fit and predict"""
        arima_model, boosted_model = estimator_tuple
        # Fit ARIMA model
        arima_model.fit(y[train]) # X=X.iloc[train, :]
        # Predict with ARIMA model
        arima_pred = arima_model.predict(n_periods=len(test))
        arima_pred_cap = max(min(1, arima_pred[0]), 0)
        # Calculate residuals for RF input
        arima_residuals_train = arima_pred - y[train]

        #model = xgb.XGBRegressor(objective = 'reg:absoluteerror', booster = 'gbtree', max_depth=5, steps =20, learning_rate=0.1) # 'reg:squarederror'
        # Train the model
        #model = model.fit(D_train, steps, watchlist)
        boosted_model = boosted_model.fit(X.iloc[train,1:], arima_residuals_train)
        # Predict the labels of the test set
        #preds = model.predict(D_test)
        preds = boosted_model.predict(X.iloc[test,1:])
        # Overall prediction residuals = pred - true <=> true = pred - residuals
        overall_pred = np.array(max(min(1, arima_pred[0] - preds), 0)) # make sure it is in [0;1]

        return overall_pred, test, arima_pred_cap, boosted_model.feature_importances_ #arima_residuals_test

    y, X = indexable(y, X)
    y = check_endog(y, copy=False, preserve_series=True)
    cv = check_cv(cv, initial)
    avgfunc = _check_averaging(averaging)

    if cv.step > cv.horizon:
        raise ValueError("CV step cannot be > CV horizon, or there will be a gap in predictions between folds")

    prediction_blocks = [
        _fit_and_predict(fold,
                         estimator,
                         y,
                         X,
                         train=train,
                         test=test,
                         verbose=verbose,)  # TODO: fit params?
        for fold, (train, test) in enumerate(cv.split(y, X))]

    pred_matrix = np.ones((y.shape[0], len(prediction_blocks))) * np.nan
    arima_pred = []
    feature_importances = np.zeros((211,))
    for i, (pred_block, test_indices, arima_block, feature_importance) in enumerate(prediction_blocks):
        pred_matrix[test_indices, i] = pred_block
        arima_pred.append(arima_block)
        feature_importances += feature_importance


    if return_raw_predictions:
        predictions = np.ones((y.shape[0], cv.horizon)) * np.nan
        for pred_block, test_indices in prediction_blocks:
            predictions[test_indices[0]] = pred_block
        return predictions

    test_mask = ~(np.isnan(pred_matrix).all(axis=1))
    predictions = pred_matrix[test_mask]



    # Calculate CV score
    cv_scores = []
    cv_scores_arima = []
    for fold, (train, test) in enumerate(cv.split(y, X)):
        fold_predictions = pred_matrix[test, fold]
        fold_score = float(abs(y[test] - fold_predictions))
        fold_arima_score = float(abs(y[test] - arima_pred[fold]))
        cv_scores.append(fold_score)
        cv_scores_arima.append(fold_arima_score)

    # Compute overall CV score
    full_score = np.mean(cv_scores)
    arima_score = np.mean(cv_scores_arima)

    return avgfunc(predictions, axis=1), np.array(arima_pred), full_score,  arima_score, cv_scores, cv_scores_arima, feature_importances


In [8]:
import sys
import pandas as pd
import pmdarima as pm
import xgboost as xgb
import ast
import catboost as cb
import warnings
warnings.filterwarnings('ignore')

def HPC_SARIMA_Boosted(strækning = 1, station = 0, model_no = 0, no_preds=30): #sys.argv[1] sys.argv[2]
    # Extract data
    data = pd.read_csv("Data/Cleaned_data.csv")
    y = data[(data['visualiseringskode'] == strækning) & (data['station'] == station)]['togpunktlighed'].values
    X = data[(data['visualiseringskode'] == strækning) & (data['station'] == station)].iloc[:,1:]

    # Best model parameters for ARIMA
    model_params_arima = pd.read_csv('Data/Best_model_parameters_SARIMA_strækning_station.csv')
    best_arima_params = ast.literal_eval(model_params_arima[model_params_arima['Key'] == str((int(strækning), int(station)))]['Values'][0])

    # Define models
    arima_model = pm.arima.ARIMA(order = best_arima_params[0], seasonal_order=best_arima_params[1])
    if model_no: # 0=Xgboost, 1=Catboost
        boosted_model = cb.CatBoostRegressor(objective = 'MAE', iterations=20, learning_rate=0.1, max_depth=5, verbose=0)  
    boosted_model = xgb.XGBRegressor(objective = 'reg:absoluteerror', booster='gbtree', steps=20, learning_rate=0.1, max_depth=5) 

    # Expanding Window CV
    initial_start = y.shape[0] - no_preds
    pred_full, pred_arima, error_full, error_arima, cv_score_full, cv_score_arima, feature_importances = custom_cross_val_predict((arima_model, boosted_model), y, X, cv=None, verbose=1, averaging="mean", return_raw_predictions=False, initial=initial_start)
    result_dictionary = {'Predictions_full': pred_full, 'Predictions_arima': pred_arima, 'Error_full': error_full, 'Error_arima': error_arima, 'CV_score_full': cv_score_full, 
                  'CV_score_arima': cv_score_arima}
    
    # Save results in .csv's
    result_df = pd.DataFrame(result_dictionary)
    fi_df = pd.DataFrame(feature_importances, columns=['Feature_importances'])
    result_df.to_csv(f'Results/({strækning}, {station})_SARIMA_Boosted{model_no}_results.csv', index=False)
    fi_df.to_csv(f'Results/({strækning}, {station})_SARIMA_Boosted{model_no}_feature_importances.csv', index=False)

In [9]:
HPC_SARIMA_Boosted(strækning = 1, station = 0, model_no=1)

# Final

In [ ]:
from pmdarima.model_selection import RollingForecastCV
import numpy as np

# Customized cross validation with rolling window and XGboost
def custom_cross_val_predict(estimator, y, X=None, cv=None, verbose=0, averaging="mean", return_raw_predictions=False, initial=2555):
    """Generate cross-validated estimates for each input data point
    
    Parameters
    ----------
    estimator : tuple
        A tuple containing two estimators. The first estimator should be the ARIMA model
        and the second one should be the Random Forest model.

    y : array-like or iterable, shape=(n_samples,)
        The time-series array.

    X : array-like, shape=[n_obs, n_vars], optional (default=None)
        An optional 2-d array of exogenous variables.

    cv : BaseTSCrossValidator or None, optional (default=None)
        An instance of cross-validation. If None, will use a RollingForecastCV.
        Note that for cross-validation predictions, the CV step cannot exceed
        the CV horizon, or there will be a gap between fold predictions.

    verbose : integer, optional
        The verbosity level.

    averaging : str or callable, one of ["median", "mean"] (default="mean")
        Unlike normal CV, time series CV might have different folds (windows)
        forecasting the same time step. After all forecast windows are made,
        we build a matrix of y x n_folds, populating each fold's forecasts like
        so::

            nan nan nan  # training samples
            nan nan nan
            nan nan nan
            nan nan nan
              1 nan nan  # test samples
              4   3 nan
              3 2.5 3.5
            nan   6   5
            nan nan   4

        We then average each time step's forecasts to end up with our final
        prediction results.

    return_raw_predictions : bool (default=False)
        If True, raw predictions are returned instead of averaged ones.
        This results in a y x h matrix. For example, if h=3, and step=1 then:

            nan nan nan # training samples
            nan nan nan
            nan nan nan
            nan nan nan
            1   4   2   # test samples
            2   5   7
            8   9   1
            nan nan nan
            nan nan nan

        First column contains all one-step-ahead-predictions, second column all
        two-step-ahead-predictions etc. Further metrics can then be calculated
        as desired.

    Returns
    -------
    predictions : array-like, shape=(n_samples,)
        The predicted values.

    """

    def indexable(*iterables):
        """Internal utility to handle input types"""
        results = []
        for iterable in iterables:
            if not hasattr(iterable, "__iter__"):
                raise ValueError("Input {!r} is not indexable".format(iterable))
            results.append(iterable)
        return results

    def check_cv(cv, initial = 2555):
        """Internal utility to check cv"""
        if cv is None:
            cv = RollingForecastCV(initial=initial, step=1, h=1)
        return cv

    def check_endog(y, copy=True, preserve_series=False):
        """Internal utility to check endogenous variable"""
        from pmdarima.utils import check_endog
        return check_endog(y, copy=copy, preserve_series=preserve_series)

    def _check_averaging(averaging):
        """Internal utility to check averaging"""
        if averaging == "mean":
            return np.nanmean
        elif averaging == "median":
            return np.nanmedian
        elif callable(averaging):
            return averaging
        else:
            raise ValueError("Unknown averaging method: {}".format(averaging))

    def _fit_and_predict(fold, estimator_tuple, y, X, train, test, verbose=1):
        """Internal utility to fit and predict"""
        arima_model, boosted_model = estimator_tuple
        # Fit ARIMA model
        arima_model.fit(y[train]) # X=X.iloc[train, :]
        # Predict with ARIMA model
        arima_pred = arima_model.predict(n_periods=len(test))
        arima_pred_cap = max(min(1, arima_pred[0]), 0)
        # Calculate residuals for RF input
        arima_residuals_train = arima_pred - y[train]

        #model = xgb.XGBRegressor(objective = 'reg:absoluteerror', booster = 'gbtree', max_depth=5, steps =20, learning_rate=0.1) # 'reg:squarederror'
        # Train the model
        #model = model.fit(D_train, steps, watchlist)
        boosted_model = boosted_model.fit(X.iloc[train,1:], arima_residuals_train)
        # Predict the labels of the test set
        #preds = model.predict(D_test)
        preds = boosted_model.predict(X.iloc[test,1:])
        # Overall prediction residuals = pred - true <=> true = pred - residuals
        overall_pred = np.array(max(min(1, arima_pred[0] - preds), 0)) # make sure it is in [0;1]

        return overall_pred, test, arima_pred_cap, boosted_model.feature_importances_ #arima_residuals_test

    y, X = indexable(y, X)
    y = check_endog(y, copy=False, preserve_series=True)
    cv = check_cv(cv, initial)
    avgfunc = _check_averaging(averaging)

    if cv.step > cv.horizon:
        raise ValueError("CV step cannot be > CV horizon, or there will be a gap in predictions between folds")

    prediction_blocks = [
        _fit_and_predict(fold,
                         estimator,
                         y,
                         X,
                         train=train,
                         test=test,
                         verbose=verbose,)  # TODO: fit params?
        for fold, (train, test) in enumerate(cv.split(y, X))]

    pred_matrix = np.ones((y.shape[0], len(prediction_blocks))) * np.nan
    arima_pred = []
    feature_importances = np.zeros((211,))
    for i, (pred_block, test_indices, arima_block, feature_importance) in enumerate(prediction_blocks):
        pred_matrix[test_indices, i] = pred_block
        arima_pred.append(arima_block)
        feature_importances += feature_importance


    if return_raw_predictions:
        predictions = np.ones((y.shape[0], cv.horizon)) * np.nan
        for pred_block, test_indices in prediction_blocks:
            predictions[test_indices[0]] = pred_block
        return predictions

    test_mask = ~(np.isnan(pred_matrix).all(axis=1))
    predictions = pred_matrix[test_mask]



    # Calculate CV score
    cv_scores = []
    cv_scores_arima = []
    for fold, (train, test) in enumerate(cv.split(y, X)):
        fold_predictions = pred_matrix[test, fold]
        fold_score = float(abs(y[test] - fold_predictions))
        fold_arima_score = float(abs(y[test] - arima_pred[fold]))
        cv_scores.append(fold_score)
        cv_scores_arima.append(fold_arima_score)

    # Compute overall CV score
    full_score = np.mean(cv_scores)
    arima_score = np.mean(cv_scores_arima)

    return avgfunc(predictions, axis=1), np.array(arima_pred), full_score,  arima_score, cv_scores, cv_scores_arima, feature_importances

import sys
import pandas as pd
import pmdarima as pm
import xgboost as xgb
import ast
import catboost as cb
import warnings
warnings.filterwarnings('ignore')

def HPC_SARIMA_Boosted(strækning = sys.argv[1], station = sys.argv[2], model_no = sys.argv[3], no_preds=sys.argv[4]): #sys.argv[1] sys.argv[2]
    # Extract data
    data = pd.read_csv("Data/Cleaned_data.csv")
    y = data[(data['visualiseringskode'] == strækning) & (data['station'] == station)]['togpunktlighed'].values
    X = data[(data['visualiseringskode'] == strækning) & (data['station'] == station)].iloc[:,1:]

    # Best model parameters for ARIMA
    model_params_arima = pd.read_csv('Data/Best_model_parameters_SARIMA_strækning_station.csv')
    best_arima_params = ast.literal_eval(model_params_arima[model_params_arima['Key'] == str((int(strækning), int(station)))]['Values'][0])

    # Define models
    arima_model = pm.arima.ARIMA(order = best_arima_params[0], seasonal_order=best_arima_params[1])
    if model_no: # 0=Xgboost, 1=Catboost
        boosted_model = cb.CatBoostRegressor(objective = 'MAE', iterations=20, learning_rate=0.1, max_depth=5, verbose=0)  
    boosted_model = xgb.XGBRegressor(objective = 'reg:absoluteerror', booster='gbtree', steps=20, learning_rate=0.1, max_depth=5) 

    # Expanding Window CV
    initial_start = y.shape[0] - no_preds
    pred_full, pred_arima, error_full, error_arima, cv_score_full, cv_score_arima, feature_importances = custom_cross_val_predict((arima_model, boosted_model), y, X, cv=None, verbose=1, averaging="mean", return_raw_predictions=False, initial=initial_start)
    result_dictionary = {'Predictions_full': pred_full, 'Predictions_arima': pred_arima, 'Error_full': error_full, 'Error_arima': error_arima, 'CV_score_full': cv_score_full, 
                  'CV_score_arima': cv_score_arima}
    
    # Save results in .csv's
    result_df = pd.DataFrame(result_dictionary)
    fi_df = pd.DataFrame(feature_importances, columns=['Feature_importances'])
    result_df.to_csv(f'Results/({strækning}, {station})_SARIMA_Boosted{model_no}_results.csv', index=False)
    fi_df.to_csv(f'Results/({strækning}, {station})_SARIMA_Boosted{model_no}_feature_importances.csv', index=False)

In [12]:
import glob,subprocess
import numpy as np  # Ensure numpy is imported
combinations = [[1,0], [1,1], [2,0]]
cmd = '''hi
safa'''
for straekning, station in combinations:
    cmdi = cmd + f' {straekning} {station} 0 30'
    proc = subprocess.Popen('bsub ', stdin=subprocess.PIPE, shell=True)
    proc.communicate(bytes(cmdi, encoding='UTF-8'))

/bin/sh: bsub: command not found
/bin/sh: bsub: command not found
/bin/sh: bsub: command not found


In [2]:
import ast
ast.literal_eval('[(3, 1, 5), (0, 0, 0, 0)]')[0]

(3, 1, 5)